In [ ]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, normalized_mutual_info_score
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),#244x224
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) #c/std
])

In [ ]:

dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
indices = torch.arange(5000)
dataloader = torch.utils.data.DataLoader(torch.utils.data.Subset(dataset, indices), batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:08<00:00, 20.2MB/s]


In [ ]:
#Feature Extraction
class AdaptiveCNN:
    def __init__(self):
        self.base_model = torchvision.models.resnet18(pretrained=True).to(device)
        self.base_model.eval()

        self.available_layers = list(self.base_model.children())[:-1]#All whto fc

    def get_features(self, layer_index):
        #extractor
        model_cut = nn.Sequential(*self.available_layers[:layer_index+1]).to(device)
        features = []
        labels_true = []

        with torch.no_grad():
            for imgs, lbls in dataloader:
                output = model_cut(imgs.to(device))
                output = torch.flatten(output, 1)
                features.append(output.cpu().numpy())
                labels_true.append(lbls.numpy())

        return np.concatenate(features), np.concatenate(labels_true)

In [ ]:
#Enviorment

def q_bin(sil):
    if sil < 0.2:
        return 0
    elif sil < 0.4:
        return 1
    else:
        return 2


class ClusteringEnv:
    def __init__(self):
        self.cnn = AdaptiveCNN()
        self.k_options = [2, 5, 10, 15, 20] #k action spave
        self.layer_options = [4, 5, 6, 7, 8] #layers
        self.reset()




    def reset(self):
        self.state = [2, 2, 1] #kI, lI, qB
        return self.state

    def step(self, action):

        #Actions
        if action == 0 and self.state[0] < len(self.k_options)-1:
           self.state[0] += 1 #k++
        elif action == 1 and self.state[0] > 0:
           self.state[0] -= 1 #k--
        elif action == 2 and self.state[1] < len(self.layer_options)-1:
          self.state[1] +=1 #l++
        elif action == 3 and self.state[1] > 0:
           self.state[1] -= 1 #l--
        elif action == 4:
           pass #doNothing


        k = self.k_options[self.state[0]]
        layer = self.layer_options[self.state[1]]

        #clustering
        feats, labels_true = self.cnn.get_features(layer)
        kmeans = KMeans(n_clusters=k, n_init=5, random_state=42)
        preds = kmeans.fit_predict(feats)


        sil = silhouette_score(feats, preds)
        nmi = normalized_mutual_info_score(labels_true, preds)
        q_bin = q_bin(sil)

        self.state = [self.state[0], self.state[1], q_bin]
       # k_penalty = -abs(k-10)/10


        #k_prior = np.exp(-((k - 10) ** 2) / 20)
        #reward = sil + 0.5 *nmi +0.3 * k_penalty ///b
        #reward = sil + 0.5 * nmi + 0.2 * k_prior ///b
        reward = sil + nmi


        #new State
        return self.state, reward, nmi

In [ ]:
#Q learnig training


#Ag/Env innit
env = ClusteringEnv()
q_table = np.zeros((5, 5, 3, 5)) #kI,lI, qB, a
alpha = 0.1
gamma = 0.9

episode = 30
epsilon = 0.4
#epsilon = max(0.05, 0.4 * (0.99 ** episode))



results = []

print("\n Adaptive optimization")
for episode in tqdm(range(episode)):
    state = env.reset()#k=10,l=6 qb=mediu (ma mai gandesc daca le fac random)
    for step in range(5):
        #e greedy
        if random.random() < epsilon:
            action = random.randint(0, 4)
        else:
            action = np.argmax(q_table[state[0], state[1], state[2]])

        next_state, reward, nmi = env.step(action)

        # Update Q-Value
        q_vechi = q_table[state[0], state[1], state[2], action]
        next_max = np.max(q_table[next_state[0], next_state[1], next_state[2]])
        q_table[state[0], state[1], state[2], action] = q_vechi + alpha * (reward + gamma * next_max - q_vechi)

        state = next_state
        results.append((reward, nmi, env.k_options[state[0]], env.layer_options[state[1]]))


 Adaptive optimization


100%|██████████| 30/30 [1:03:00<00:00, 126.02s/it]


In [ ]:
#Results

best_config = max(results, key=lambda x: x[0])
print(f"\n Results")
print(f"Nr cluster = {best_config[2]}")
print(f"Index layer = {best_config[3]}")
print(f"Silhouette = {best_config[0]:.4f}")
print(f"NMI = {best_config[1]:.4f}")


 Results
Nr cluster = 10
Index layer = 6
Silhouette = 0.3726
NMI = 0.3692


In [ ]:

print("\n Rezultate statice pe K means")


cnn_static = AdaptiveCNN()


STATIC_K = besti_config[2]#k=10
STATIC_LAYER = besti_config[3]#l=6


features_static, labels_static = cnn_static.get_features(STATIC_LAYER)


kmeans_static = KMeans(
    n_clusters=STATIC_K,
    n_init=10,
    random_state=42
)

preds_static = kmeans_static.fit_predict(features_static)


sil_static = silhouette_score(features_static, preds_static)
nmi_static = normalized_mutual_info_score(labels_static, preds_static)

print("\n Rezultate k means")
print(f"K = {STATIC_K}")
print(f"Layer CNN = {STATIC_LAYER}")
print(f"Silhouette Score = {sil_static:.4f}")
print(f"NMI Score = {nmi_static:.4f}")



 Rezultate statice pe K means


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



 Rezultate k means
K = 10
Layer CNN = 6
Silhouette Score = 0.0062
NMI Score = 0.3662
